In [ ]:
# Setup: Ensure all required packages are available
import sys
import subprocess

def install_if_missing(package_name, import_name=None):
    """Install package if not available."""
    if import_name is None:
        import_name = package_name
    
    try:
        __import__(import_name)
        print(f"✓ {package_name} is available")
    except ImportError:
        print(f"⚠ {package_name} not found. Installing...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
        print(f"✓ {package_name} installed successfully")

# Check and install critical packages
packages_to_check = [
    ('pandas', 'pandas'),
    ('numpy', 'numpy'),
    ('joblib', 'joblib')
]

print("Checking required packages...")
for pkg, imp in packages_to_check:
    install_if_missing(pkg, imp)

print("\n✅ All required packages are ready!")

# Funcao de Predicao - X-Health Credit Risk

Guia pratico de uso da funcao `predict_default()` para inferir probabilidade de default.

**Pipeline do modelo:** preprocessor (ColumnTransformer) -> XGBoost classifier

**Feature engineering automatico:** a funcao `engineer_features()` cria features derivadas
(ratios financeiros, flags de risco) automaticamente a partir dos dados brutos.

**Tratamento de missing values:**
- Categoricas: valores ausentes tratados como `'unknown'` (categoria propria utilizada pelo modelo)
- Numericas: valores ausentes tratados como `NaN` (imputer usa mediana do treino)

**Nota:** A variavel `month` e utilizada como feature categorica (OneHotEncoder).
Correlacao com a taxa de default foi identificada no EDA, indicando sazonalidade.

O pipeline recebe dados brutos e faz feature engineering + preprocessing internamente.

In [17]:
import pandas as pd
import numpy as np
import joblib
import json
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

# Carregar metadata
with open('../models/model_metadata.json', 'r') as f:
    metadata = json.load(f)

# Carregar modelo (pipeline: preprocessor -> classifier)
model_path = list(Path('../models/').glob('model_*.pkl'))[0]
model = joblib.load(model_path)

# Features e thresholds do metadata
ALL_FEATURES = metadata['numeric_features'] + metadata['high_cardinality_features'] + metadata['low_cardinality_features']
CATEGORICAL_FEATURES = metadata['high_cardinality_features'] + metadata['low_cardinality_features']
DERIVED_FEATURES = metadata.get('derived_features', [])
RAW_FEATURES = [f for f in ALL_FEATURES if f not in DERIVED_FEATURES]

THRESHOLD_F1 = metadata.get('optimal_threshold_f1', 0.5)
THRESHOLD_F2 = metadata.get('optimal_threshold_f2', 0.3)
THRESHOLD_F05 = metadata.get('optimal_threshold_f05', 0.7)


def engineer_features(df):
    """
    Cria features derivadas a partir dos dados brutos.
    Mesma funcao definida em 2_model_pipeline.ipynb.
    """
    df = df.copy()

    if 'valor_vencido' in df.columns and 'valor_quitado' in df.columns:
        df['ratio_vencido_quitado'] = df['valor_vencido'] / (df['valor_quitado'] + 1)

    if 'valor_total_pedido' in df.columns and 'valor_quitado' in df.columns:
        df['ratio_pedido_historico'] = df['valor_total_pedido'] / (df['valor_quitado'] + 1)

    if 'valor_por_vencer' in df.columns and 'valor_vencido' in df.columns:
        df['exposicao_total'] = df['valor_por_vencer'] + df['valor_vencido']

    if 'quant_protestos' in df.columns:
        df['tem_protestos'] = (df['quant_protestos'] > 0).astype(int)

    if 'quant_acao_judicial' in df.columns:
        df['tem_acao_judicial'] = (df['quant_acao_judicial'] > 0).astype(int)

    return df


def to_categorical(x):
    """Converte valor para string categorica. Trata missing como 'unknown'."""
    if pd.isna(x) or x is None or x == 'missing' or x == '':
        return 'unknown'
    try:
        # Numerico -> int -> str (ex: 3.0 -> '3', para month)
        return str(int(float(x)))
    except (ValueError, TypeError):
        return str(x)  # String normal (ex: 'Boleto')


print(f"Modelo: {metadata['model_name']}")
print(f"ROC-AUC: {metadata['test_roc_auc']:.4f} | F1: {metadata['test_f1']:.4f}")
print(f"CV ROC-AUC: {metadata['cv_roc_auc_mean']:.4f} (+/- {metadata['cv_roc_auc_std']:.4f})")
print(f"\nThresholds: F1={THRESHOLD_F1:.4f} | F2={THRESHOLD_F2:.4f} | F0.5={THRESHOLD_F05:.4f}")
print(f"\nFeatures brutas ({len(RAW_FEATURES)}): usuario fornece")
print(f"Features derivadas ({len(DERIVED_FEATURES)}): calculadas por engineer_features()")
print(f"Total features pipeline ({len(ALL_FEATURES)}): {len(RAW_FEATURES)} brutas + {len(DERIVED_FEATURES)} derivadas")
print(f"\nFeatures categoricas: {CATEGORICAL_FEATURES}")

Modelo: XGBoost
ROC-AUC: 0.9168 | F1: 0.6542
CV ROC-AUC: 0.9175 (+/- 0.0018)

Thresholds: F1=0.6413 | F2=0.4182 | F0.5=0.8022

Features brutas (20): usuario fornece
Features derivadas (5): calculadas por engineer_features()
Total features pipeline (25): 20 brutas + 5 derivadas

Features categoricas: ['tipo_sociedade', 'atividade_principal', 'forma_pagamento', 'opcao_tributaria', 'month']


## Funcao de Predicao

- `threshold=None` usa o otimo F1 do metadata
- Aceita apenas features brutas (19) — features derivadas sao calculadas automaticamente
- Missing values: categoricas → `'unknown'`, numericas → `NaN` (imputer trata)
- O pipeline aplica encoding e scaling internamente
- Retorna: `{default, probability, confidence, threshold_used}`

In [18]:
def predict_default(input_dict, threshold=None):
    """
    Predicao de default com feature engineering automatico.

    Aceita features brutas (sem derivadas) - engineer_features() cria automaticamente.
    Missing values: categoricas -> 'unknown', numericas -> NaN (imputer usa mediana).
    Categoricas recebidas como numero (ex: month=3) sao convertidas para string ('3').
    """
    if threshold is None:
        threshold = THRESHOLD_F1
    try:
        input_df = pd.DataFrame([input_dict])

        # Tratar missing values e tipos
        for col in input_df.columns:
            if col in CATEGORICAL_FEATURES:
                # Categoricas: converter para string, missing -> 'unknown'
                input_df[col] = input_df[col].apply(to_categorical)
            else:
                # Numericas: 'missing' -> NaN, garantir tipo numerico
                input_df[col] = pd.to_numeric(
                    input_df[col].replace('missing', np.nan), errors='coerce'
                )

        # Feature engineering automatico (cria features derivadas)
        input_df = engineer_features(input_df)

        # Preencher features nao fornecidas
        for feature in ALL_FEATURES:
            if feature not in input_df.columns:
                input_df[feature] = 'unknown' if feature in CATEGORICAL_FEATURES else np.nan

        input_df = input_df[ALL_FEATURES]
        prob = float(model.predict_proba(input_df)[0, 1])
        pred = 1 if prob >= threshold else 0
        return {
            "default": pred,
            "probability": round(prob, 4),
            "confidence": round(max(prob, 1 - prob), 4),
            "threshold_used": round(threshold, 4)
        }
    except Exception as e:
        return {"error": str(e)}


def predict_default_batch(input_list, threshold=None):
    """Predicao em batch. Mesma interface, aceita lista de dicts."""
    return [predict_default(d, threshold) for d in input_list]

## Exemplos de Uso

Os exemplos usam apenas **features brutas** (19). Features derivadas (ratios, flags)
sao calculadas automaticamente pela funcao `engineer_features()`.

Missing values em categoricas sao tratados como `'unknown'` (nao NaN).

In [19]:
# Exemplo 1: Cliente bom pagador (apenas features brutas)
# Features derivadas sao calculadas automaticamente por engineer_features()
cliente_bom = {
    # Numericas - historico
    "default_3months": 0,
    "ioi_36months": 95,
    "ioi_3months": 98,
    # Numericas - valores financeiros
    "valor_por_vencer": 30000,
    "valor_vencido": 0,
    "valor_quitado": 500000,
    "quant_protestos": 0,
    "valor_protestos": 0,
    "quant_acao_judicial": 0,
    "acao_judicial_valor": 0,
    "participacao_falencia_valor": 0,
    "dividas_vencidas_valor": 0,
    "dividas_vencidas_qtd": 0,
    "falencia_concordata_qtd": 0,
    "valor_total_pedido": 25000,
    # Categoricas (month incluido — correlacao com default no EDA)
    "month": 6,
    "tipo_sociedade": "Limitada",
    "atividade_principal": "Comercio varejista",
    "forma_pagamento": "Boleto",
    "opcao_tributaria": "simples nacional",
}

result = predict_default(cliente_bom)
print("=== Cliente Bom Pagador ===")
print(f"Default: {result['default']} | Prob: {result['probability']:.2%} | Confianca: {result['confidence']:.2%} | Threshold: {result['threshold_used']}")

=== Cliente Bom Pagador ===
Default: 0 | Prob: 4.19% | Confianca: 95.81% | Threshold: 0.6413


In [20]:
# Exemplo 2: Cliente alto risco (apenas features brutas)
# Features derivadas (ratio_vencido_quitado, tem_protestos, etc.)
# sao calculadas automaticamente por engineer_features()
cliente_risco = {
    # Numericas - historico
    "default_3months": 8,
    "ioi_36months": 15,
    "ioi_3months": 4,
    # Numericas - valores financeiros
    "valor_por_vencer": 350000,
    "valor_vencido": 12000,
    "valor_quitado": 1800000,
    "quant_protestos": 5,
    "valor_protestos": 45000,
    "quant_acao_judicial": 0,
    "acao_judicial_valor": 0,
    "participacao_falencia_valor": 0,
    "dividas_vencidas_valor": 0,
    "dividas_vencidas_qtd": 0,
    "falencia_concordata_qtd": 0,
    "valor_total_pedido": 8500,
    # Categoricas
    "month": 11,
    "tipo_sociedade": "sociedade empresaria limitada",
    "atividade_principal": "loja de departamentos",
    "forma_pagamento": "30/60/90/120",
    "opcao_tributaria": "simples nacional",
}

result = predict_default(cliente_risco)
print("=== Cliente Alto Risco ===")
print(f"Default: {result['default']} | Prob: {result['probability']:.2%} | Confianca: {result['confidence']:.2%} | Threshold: {result['threshold_used']}")

=== Cliente Alto Risco ===
Default: 1 | Prob: 75.86% | Confianca: 75.86% | Threshold: 0.6413


## Comparacao de Thresholds

Thresholds carregados automaticamente do `model_metadata.json`.

In [21]:
# Comparar thresholds usando o cliente de risco como referencia
thresholds = [
    ("F2 (max recall)", THRESHOLD_F2),
    ("F1 (balanceado)", THRESHOLD_F1),
    ("F0.5 (max precision)", THRESHOLD_F05),
]

print("=== Comparacao de Thresholds ===")
print(f"Probabilidade do cliente: {predict_default(cliente_risco)['probability']:.4f}\n")

for label, t in thresholds:
    r = predict_default(cliente_risco, threshold=t)
    status = "DEFAULT" if r["default"] == 1 else "PAGO"
    print(f"  {label:25s} | threshold={t:.4f} | {status:7s} | prob={r['probability']:.4f}")


=== Comparacao de Thresholds ===
Probabilidade do cliente: 0.7586

  F2 (max recall)           | threshold=0.4182 | DEFAULT | prob=0.7586
  F1 (balanceado)           | threshold=0.6413 | DEFAULT | prob=0.7586
  F0.5 (max precision)      | threshold=0.8022 | PAGO    | prob=0.7586


## Predicao em Batch

In [22]:
# Batch com 3 perfis distintos
# 3o perfil: dados vazios — categoricas preenchidas com 'unknown', numericas com NaN
batch = [cliente_bom, cliente_risco, {}]
labels = ["Bom pagador", "Alto risco", "Dados vazios"]

results = predict_default_batch(batch)

print("=== Predicao em Batch ===")
for label, r in zip(labels, results):
    print(f"  {label:15s} | default={r['default']} | prob={r['probability']:.4f}")

=== Predicao em Batch ===
  Bom pagador     | default=0 | prob=0.0419
  Alto risco      | default=1 | prob=0.7586
  Dados vazios    | default=0 | prob=0.0996


## Validacao com Dados Reais

In [23]:
# Validacao com amostra do dataset original
df = pd.read_csv('../_data/dataset_2021-5-26-10-14.csv', sep='\t', encoding='utf-8')
# predict_default() trata missing values e tipos internamente:
# - Categoricas: 'missing'/NaN -> 'unknown', numerico -> str (ex: month 3 -> '3')
# - Numericas: 'missing'/NaN -> NaN (imputer usa mediana do treino)
# - Features derivadas: calculadas automaticamente por engineer_features()

np.random.seed(15)
sample = df.sample(10)

acertos = 0
print("=== Validacao (10 registros aleatorios) ===")
for idx, row in sample.iterrows():
    real = int(row['default'])
    pred = predict_default(row.drop('default').to_dict())
    match = "OK" if real == pred["default"] else "ERRO"
    acertos += 1 if match == "OK" else 0
    print(f"  #{idx:5d} | real={real} pred={pred['default']} prob={pred['probability']:.4f} | {match}")

print(f"\nAcuracia na amostra: {acertos}/10 ({acertos*10}%)")

=== Validacao (10 registros aleatorios) ===
  #53174 | real=0 pred=0 prob=0.2159 | OK
  #60041 | real=0 pred=0 prob=0.3739 | OK
  #34669 | real=0 pred=0 prob=0.2044 | OK
  #106851 | real=1 pred=1 prob=0.9930 | OK
  #116590 | real=1 pred=0 prob=0.6170 | ERRO
  #50262 | real=0 pred=0 prob=0.2337 | OK
  # 1490 | real=0 pred=0 prob=0.0385 | OK
  # 8500 | real=0 pred=0 prob=0.2864 | OK
  #91411 | real=0 pred=0 prob=0.2221 | OK
  #56374 | real=0 pred=0 prob=0.0313 | OK

Acuracia na amostra: 9/10 (90%)


## Exportar Script Standalone

In [25]:
script = '''
"""predict.py - Predicao de default (X-Health Credit Risk).

Feature engineering automatico: features derivadas calculadas internamente.
Missing values: categoricas -> 'unknown', numericas -> NaN (imputer usa mediana).
month e tratado como categorica (correlacao com default no EDA).
"""
import pandas as pd, numpy as np, joblib, json
from pathlib import Path

model = joblib.load(list(Path('models/').glob('model_*.pkl'))[0])
with open('models/model_metadata.json') as f: metadata = json.load(f)

ALL_FEATURES = metadata['numeric_features'] + metadata['high_cardinality_features'] + metadata['low_cardinality_features']
CATEGORICAL_FEATURES = metadata['high_cardinality_features'] + metadata['low_cardinality_features']
DERIVED_FEATURES = metadata.get('derived_features', [])
THRESHOLD_F1 = metadata.get('optimal_threshold_f1', 0.5)


def engineer_features(df):
    df = df.copy()
    if 'valor_vencido' in df.columns and 'valor_quitado' in df.columns:
        df['ratio_vencido_quitado'] = df['valor_vencido'] / (df['valor_quitado'] + 1)
    if 'valor_total_pedido' in df.columns and 'valor_quitado' in df.columns:
        df['ratio_pedido_historico'] = df['valor_total_pedido'] / (df['valor_quitado'] + 1)
    if 'valor_por_vencer' in df.columns and 'valor_vencido' in df.columns:
        df['exposicao_total'] = df['valor_por_vencer'] + df['valor_vencido']
    if 'quant_protestos' in df.columns:
        df['tem_protestos'] = (df['quant_protestos'] > 0).astype(int)
    if 'quant_acao_judicial' in df.columns:
        df['tem_acao_judicial'] = (df['quant_acao_judicial'] > 0).astype(int)
    return df


def to_categorical(x):
    if pd.isna(x) or x is None or x == 'missing' or x == '':
        return 'unknown'
    try:
        return str(int(float(x)))
    except (ValueError, TypeError):
        return str(x)


def predict_default(input_dict, threshold=None):
    if threshold is None: threshold = THRESHOLD_F1
    df = pd.DataFrame([input_dict])
    for col in df.columns:
        if col in CATEGORICAL_FEATURES:
            df[col] = df[col].apply(to_categorical)
        else:
            df[col] = pd.to_numeric(df[col].replace('missing', np.nan), errors='coerce')
    df = engineer_features(df)
    for f in ALL_FEATURES:
        if f not in df.columns:
            df[f] = 'unknown' if f in CATEGORICAL_FEATURES else np.nan
    prob = float(model.predict_proba(df[ALL_FEATURES])[0, 1])
    return {"default": int(prob >= threshold), "probability": round(prob, 4), "threshold_used": round(threshold, 4)}

if __name__ == '__main__':
    print(predict_default({"ioi_3months": 3, "valor_vencido": 125000, "valor_total_pedido": 35000, "month": 6}))
'''

with open('../scripts/predict.py', 'w') as f:
    f.write(script)
print("Salvo em scripts/predict.py")

Salvo em scripts/predict.py
